In [44]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.model.rerank_model_finetune import RerankModelFinetune
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere_api=Cohere(setting)
bert=Bert_Extract(setting)
rerank_model_fintuned=RerankModelFinetune(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere_api,rerank_model_fintuned)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model


OSError: The paging file is too small for this operation to complete. (os error 1455)

In [ ]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.model.rerank_model_finetune import RerankModelFinetune
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere_api=Cohere(setting)
bert=Bert_Extract(setting)
rerank_model_fintuned=RerankModelFinetune(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere_api,rerank_model_fintuned)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model


## 1.Eval Gemini model

In [34]:
# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')

In [35]:

pattern = r'Xin\s+lỗi\s+bạn'  # regex cho “Xin lỗi bạn” (có thể có nhiều khoảng trắng)

# Chỉ giữ lại những dòng không khớp pattern
df = df[~df['answer_from_gemini_rag_basic'].str.contains(pattern, regex=True, na=False)]

In [31]:
df = df[~df['answer_from_gemini_rag_final'].str.contains(r'""', regex=True, na=False)]

In [40]:

pattern = r'Xin\s+lỗi\s+bạn'  # regex cho “Xin lỗi bạn” (có thể có nhiều khoảng trắng)

# Chỉ giữ lại những dòng không khớp pattern
df = df[~df['answer_from_gemini_rag_final_extract'].str.contains(pattern, regex=True, na=False)]

In [38]:
df = df.dropna()

In [41]:
print(df.shape)

(456, 6)


In [42]:
df = df.reset_index(drop=True)

In [5]:
def Generate_Response(df_batch):
    answers = []
    for idx, question in tqdm(list(enumerate(df_batch['question'])), desc="Sinh trả lời"):
        try:
            article_doc = rag.get_gemini_response_rag_final(question)
            answers.append(article_doc)
        except Exception as e:
            print(f"\n❌ Lỗi tại index {idx} (DataFrame index = {df_batch.index[idx]}):\n{question}")
            answers.append("")
    return answers


df['answer_from_gemini_rag_final'] = Generate_Response(df)


Sinh trả lời:   0%|          | 0/564 [00:00<?, ?it/s]

Sinh trả lời:  37%|███▋      | 207/564 [1:05:14<5:52:34, 59.26s/it]

An error occurred: status_code: 502, body: 
<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>502 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>


❌ Lỗi tại index 206 (DataFrame index = 206):
Một số gia đình khi thuê người giúp việc gia đình vẫn thường than phiền khi tết đến phải cho người giúp việc tiền tàu xe, còn sợ sau tết họ lại bỏ việc. Đề nghị cho biết pháp luật quy định về vấn đề này như thế nào?


Sinh trả lời: 100%|██████████| 564/564 [2:52:13<00:00, 18.32s/it]  


In [6]:
final_output_path = './data/data_processed/final_data_system_response.csv'
df.to_csv(final_output_path, index=False)
print(f" Đã cập nhật và lưu toàn bộ vào: {final_output_path}")

 Đã cập nhật và lưu toàn bộ vào: ./data/data_processed/final_data_system_response.csv


> Configure

In [43]:
setting=Settings()

> Get embedding gemini 

In [45]:
print(df.shape)

(456, 6)


In [11]:
import google.generativeai as genai

genai.configure(api_key=gemini.key_manager.get_next_key())

def get_gemini_embedding(texts):
    try :
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=texts,
            task_type="SEMANTIC_SIMILARITY"
        )
        
        return result['embedding']
    except Exception as e:
        print("Đã xảy ra lỗi:", e)
        return [0.0] * 768

> get embedding cohere 

In [12]:
import cohere

def get_cohere_embedding(text: str, model_name: str = "embed-v4.0") -> list:
    try:
        co = cohere.ClientV2(api_key=cohere_api.key_manager.get_next_key())
        res = co.embed(
            texts=[text],
            model=model_name,
            input_type="classification",
            embedding_types=["float"],
        )

        # Trả về nhúng đầu tiên
        return res.embeddings.float[0]
    except Exception as e:
        print("Đã xảy ra lỗi:", e)
        return [0.0] * 1024  # Trả về danh sách 1024 số 0 nếu có lỗi


>Get  rouge-L & F1 Score

In [13]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge_l(pred: str, ref: str) -> float:
    try:
        score = scorer.score(ref, pred)
        return score['rougeL'].fmeasure
    except:
        return 0.0

def compute_f1(pred: str, ref: str) -> float:
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()
    common = set(pred_tokens) & set(ref_tokens)
    if not pred_tokens or not ref_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [19]:
print(df.shape)

(483, 6)


In [46]:
# df['answer_embedding_cohere'] = df['answer'].apply(get_cohere_embedding)
# df['answer_from_gemini_rag_final_cohere'] = df['answer_from_gemini_rag_final'].apply(get_cohere_embedding)


df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_gemini_rag_final_embedding'] = df['answer_from_gemini_rag_final'].apply(get_gemini_embedding)

df['cosine_similarity_gemini_rag_final'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']], 
        [row['answer_from_gemini_rag_final_embedding']]
    )[0][0],
    axis=1
)

# df['cosine_similarity_cohere'] = df.apply(
#     lambda row: cosine_similarity(
#         [row['answer_embedding_cohere']], 
#         [row['answer_from_gemini_rag_final_cohere']]
#     )[0][0],
#     axis=1


Đã xảy ra lỗi: 400 Request payload size exceeds the limit: 36000 bytes.
Đã xảy ra lỗi: 400 Request payload size exceeds the limit: 36000 bytes.


In [47]:
df['rouge_l'] = df.apply(lambda row: compute_rouge_l(row['answer_from_gemini_rag_final'], row['answer']), axis=1)
df['f1'] = df.apply(lambda row: compute_f1(row['answer_from_gemini_rag_final'], row['answer']), axis=1)


In [48]:
print(f" Trung bình Cosine (Gemini): {df['cosine_similarity_gemini_rag_final'].mean():.4f}")
print(f" Trung bình ROUGE-L: {df['rouge_l'].mean():.4f}")
print(f" Trung bình F1 Score: {df['f1'].mean():.4f}") 

 Trung bình Cosine (Gemini): 0.9038
 Trung bình ROUGE-L: 0.4171
 Trung bình F1 Score: 0.2266
